In [141]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# pdf_text = extract_text_from_pdf("mock_trial.pdf")
pdf_text = extract_text_from_pdf("Mini-Mock-Trial-State-v.-Anderson-2016.pdf")


In [142]:
def estimate_tokens(text):
    return len(text) // 4

total_tokens = estimate_tokens(pdf_text)
print(f"Estimated tokens: {total_tokens}")


Estimated tokens: 1854


In [143]:
judge_prompt = """
You are a judge presiding over a mock trial. You must adhere to the trial's context and answer questions as the judge.
The user will be roleplaying as another character in the mock trial.
"""


In [144]:
defendant_prompt = """
You are pretending to be the defendant in this mock trial. You must adhere to the trial's context and answer questions as the car owner.
The user will be roleplaying as another character in the mock trial.
"""

In [145]:
import autogen
from autogen import ConversableAgent
import os


# llm_config = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY"), "api_rate_limit": 1.0}]}
llm_config = {"config_list": [{"model": "llama3.2:latest", "api_type": "ollama", "client_host": "http://127.0.0.1:11434"}]}

    
# the main entrypoint/supervisor agent
judge_agent = ConversableAgent("judge agent", system_message=judge_prompt, llm_config=llm_config,  description="Acts as the judge in the mock trial, responsible for making rulings, interpreting the law, and ensuring the trial proceeds fairly.")

In [146]:
defendant_agent = ConversableAgent(name="defendant agent", system_message=defendant_prompt, llm_config=llm_config, description="Plays the role of the defendant, Matthew Anderson, in the mock trial, providing testimony and responding to legal questions relevant to their character.")

In [147]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
    description="Represents the human user roleplaying as the Deputy DA, directing the flow of the trial and interacting with other agents."
)

In [148]:
from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[judge_agent, defendant_agent, human_proxy],
    messages=[],
    max_round=6,
)


group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config 
)

In [149]:
# Start the Conversation with Full PDF as First Message
initial_message = f"""
I will be roleplaying as the Deputy DA in this mock trial. Here is the full trial document:

{pdf_text}

Is everyone ready?
"""

In [150]:
chat_result = human_proxy.initiate_chat(
    group_chat_manager,
    message=initial_message,
    summary_method="reflection_with_llm",
)

human_proxy (to chat_manager):


I will be roleplaying as the Deputy DA in this mock trial. Here is the full trial document:

Student Handout
Mini-Mock Trial Manual
Mini-Mock Trial State v. Anderson: introduction
Criminal Mock Trial
District Court
State )
Prosecution )
)
v. )
)
Anderson )
Defendant )
Facts
On February 7, 2003, a University of Minnesota police officer
stopped a motor vehicle traveling eastbound on University
Issue:
Avenue S.E. near Walnut Street in Minneapolis because he
believed the driver failed to move a lane away from the Did the defendant
parked squad car in violation of Minn.Stat. 169.18, subd.11, move over the required
which requires motorists to move "a lane away" from lanes when passing an
stopped emergency vehicles. The driver has been charged emergency vehicle?
with violating this law.
Minn. Stat. 169.18, subd.11 (2002) reads:
Defense:
"When approaching and before passing an
authorized emergency vehicle that is parked or Defendant Anderson
otherwise stopped o

KeyboardInterrupt: 

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",  # Choose model with sufficient token capacity
#     messages=[{"role": "system", "content": roleplay_prompt}]
# )

# print(completion.choices[0].message)
